
# __Progetto: Analisi dei Grafi__

Questo notebook contiene la costruzione e l'analisi di tre tipi di grafi:

1. **Grafo delle dipendenze di Angular-cli**:
   - Rappresenta le dipendenze del pacchetto `angular-cli`.
2. **Grafo artificiale con 400 nodi**:
   - Simula una rete con nodi aggiunti tramite un modello di preferential attachment classico.
3. **Grafo artificiale con preferential attachment basato su Random Walk** (bonus):
   - Variante che utilizza random walk per determinare le connessioni dei nuovi nodi.

L'obiettivo è confrontare le proprietà strutturali di queste reti.


# <hr style="border: 1px solid white; margin-top: 10px;">

# **Librerie**

In [ ]:
import requests, json, collections
import networkx as nx
from networkx.algorithms.smallworld import sigma, omega
import numpy as np
from collections import deque
from pyvis.network import Network
from collections import Counter
import matplotlib.pyplot as plt

# <hr style="border: 1px solid white; margin-top: 10px;">

# **Funzioni**

### Grafo Diretto

In [2]:
def trova_dipendenze(package_name):
    
    URL = f"https://registry.npmjs.org/{package_name}/latest"
    try:
        response = requests.get(URL)
        if response.status_code != 200:
            return {}
        data = response.json()
        return data.get("dependencies", {})
    except Exception as e:
        print(f"Errore con {package_name}: {e}")
        return {}

In [3]:
def bfs_with_cycle_check(seed_package):
    G = nx.DiGraph()
    coda = deque([(seed_package, 0)])  # Inizializza la coda con il seed
    visitati = set()  # Nodi completamente esplorati (neri)
    in_corso = set()  # Nodi in fase di esplorazione (grigi)

    while coda:
        nodo_corrente, livello = coda.popleft()

        # Aggiungi il nodo a in_corso (grigio) e al grafo
        in_corso.add(nodo_corrente)
        G.add_node(nodo_corrente, livello=livello)

        # Trova le dipendenze del nodo corrente
        dependencies = trova_dipendenze(nodo_corrente)
        for dep in dependencies.keys():
            if dep in in_corso:
                print("Ciclo rilevato!")
                return True  # Hai trovato un ciclo

            # Aggiungi sempre l'arco (anche verso nodi già visitati)
            G.add_edge(nodo_corrente, dep)

            # Se il nodo non è stato ancora visitato, aggiungilo alla coda
            if dep not in visitati:
                coda.append((dep, livello + 1))

        # Rimuovi il nodo da in_corso e spostalo in visitati (nero)
        in_corso.remove(nodo_corrente)
        visitati.add(nodo_corrente)

    print("Nessun ciclo rilevato!")
    return G  # Restituisce il grafo se non ci sono cicli

### Preferential Attachment

In [4]:
def aggiungi_nodi_preferential(G, nodi_artificiali, m):

    for i in range(nodi_artificiali):
        nuovo_nodo = f"new_{i}" # Nome unico per il nuovo nodo
        G.add_node(nuovo_nodo)

        # Calcola probabilità basata sui gradi attuali
        somma_gradi = 2 * G.number_of_edges()
        packages = G.nodes()

        probabilita = [G.degree(package) / somma_gradi for package in packages]
        nodi_scelti = np.random.choice(packages, size = m, replace=False, p = probabilita)


        for nodo in nodi_scelti:
            G.add_edge(nuovo_nodo, nodo)

    return G

### Random Walk

In [5]:
def aggiungi_nodi_random_walk(G, nodi_artificiali, m, p):
    for i in range(nodi_artificiali):
        nuovo_nodo = f"new_{i}"  # Nome unico per il nuovo nodo

        # Primo nodo scelto casualmente (uniforme)
        nodo_corrente = np.random.choice(list(G.nodes()))
        G.add_node(nuovo_nodo)
        
        # Insieme per tenere traccia dei nodi collegati al nuovo nodo
        nodi_collegati = set()
        nodi_collegati.add(nuovo_nodo)
        
        G.add_edge(nuovo_nodo, nodo_corrente)
        nodi_collegati.add(nodo_corrente)
        
        # Aggiungi gli altri m-1 archi
        for _ in range(m - 1):
            if np.random.rand() < p:
                # Collega a un vicino del nodo corrente (se esistono vicini validi)
                vicini = [v for v in G.neighbors(nodo_corrente) if v not in nodi_collegati]
                if vicini:
                    nodo_corrente = np.random.choice(vicini)
                    G.add_edge(nuovo_nodo, nodo_corrente)
                    nodi_collegati.add(nodo_corrente)
            else:
                # Collega a un nodo casuale valido
                nodi_non_collegati = [n for n in G.nodes() if n not in nodi_collegati]
                if nodi_non_collegati:
                    nodo_corrente = np.random.choice(nodi_non_collegati)
                    G.add_edge(nuovo_nodo, nodo_corrente)
                    nodi_collegati.add(nodo_corrente)
    
    return G

### Visualizzazione PyVis

In [6]:
def calcola_quartili(gradi):

    q1 = np.percentile(gradi, 25)
    q2 = np.percentile(gradi, 50)
    q3 = np.percentile(gradi, 75)
    return q1, q2, q3

def colore_nodo(grado, q1, q2, q3):

    if grado <= q1:
        return "gray"
    elif grado <= q2:
        return "blue"
    elif grado <= q3:
        return "purple"
    else:
        return "yellow"

In [7]:
def visualizza_interattivo_con_pyvis(G, nome_file):

    # Calcolo grado totale e quartili
    if G.is_directed():
        gradi = [grado for nodo, grado in G.out_degree()]
    else:
        gradi = [grado for nodo, grado in G.degree()]

    q1, q2, q3 = calcola_quartili(gradi)

    # Inizializza PyVis
    net = Network(
        notebook=True, 
        height="1000px", 
        width="100%", 
        directed=G.is_directed()
    )

    # Personalizza i nodi
    for nodo in G.nodes():
        if G.is_directed():
            grado = G.out_degree(nodo)
        else:
            grado = G.degree(nodo)

        size = max(0, 3.5*grado)
        color = colore_nodo(grado, q1, q2, q3)

        net.add_node(
            nodo, 
            size=size, 
            color={
                "background": color,
                "border": "black",
                "highlight" : color
            },
            borderWidth=0.5,
            title=f"Nodo: {nodo}, Grado: {grado}"
        )

    for u, v in G.edges():
        net.add_edge(u, v)
    
    net.set_options("""
var options = {
    "physics": {
        "enabled": true,
        "stabilization": {
            "enabled": true,
            "iterations": 1000
        },
        "solver": "forceAtlas2Based",
        "forceAtlas2Based": {
            "gravitationalConstant": -50,
            "centralGravity": 0.01,
            "springLength": 150,
            "springConstant": 0.005
        },
        "maxVelocity": 3,
        "minVelocity": 0.1
    },
    "interaction": {
        "navigationButtons": true
    },
    "edges": {
        "color": {
            "color": "lightgrey",
            "highlight": "blue"
        },
        "width": 0.5,
        "selectionWidth": 4
    }
}
""")

    net.show(nome_file)

### Small Worldness

In [8]:
def calcola_small_worldness(G, niter=5, nrand=10, seed=None):
    sigma_value = sigma(G, niter=niter, nrand=nrand, seed=seed)
    omega_value = omega(G, niter=niter, nrand=nrand, seed=seed)
    return {"sigma": sigma_value, "omega": omega_value}

# <hr style="border: 1px solid white; margin-top: 10px;">

# ___Costruzione Grafi___

### ***Costruzione Grafo Diretto***
Procediamo con la costruzione del grafo aciclico diretto (DAG) delle dipendenze a partire dal pacchetto seed _angular-cli_. Successivamente, il grafo ottenuto verrà salvato in un file JSON denominato _grafo_diretto.json_.

In [ ]:
nome_seed = "angular-cli"

G = bfs_with_cycle_check(nome_seed)

print(f"Totale pacchetti trovati: {G.number_of_nodes()}")
print(f"Totale archi trovati: {G.number_of_edges()}")

with open("data/grafo_diretto.json", "w") as f:
    json.dump(nx.node_link_data(G), f, indent=4)

with open("graphs/grafo_diretto.json", "w") as f:
    json.dump(nx.node_link_data(G), f, indent=4)

---

### ___Costruzione Grafo Artificiale con Preferential Attachment___
Si procede alla costruzione del _grafo indiretto_ derivato dal grafo diretto delle dipendenze. A tale grafo vengono aggiunti nodi artificiali, e il grafo risultante viene successivamente salvato in un file JSON denominato _grafo_artificiale.json_.

In [10]:
G2 = G.to_undirected()

In [ ]:
# Aggiungi nodi artificiali
nodi_artificiali = 400
m = 3 # Numero di connessioni per ogni nuovo nodo
G2 = aggiungi_nodi_preferential(G2, nodi_artificiali, m)

print(f"Grafo finale: {G2.number_of_nodes()} nodi, {G2.number_of_edges()} archi.")


In [12]:
with open("graphs/grafo_artificiale.json", "w") as f:
    json.dump(nx.node_link_data(G2), f, indent=4)

---

### ___Costruzione Grafo Artificiale con Random Walk___
_Viene utilizzata una variante del modello di preferential attachment basata su random walk._

In [13]:
G3 = G.to_undirected()

In [ ]:
# Aggiungi nodi usando il random walk
G3 = aggiungi_nodi_random_walk(G3, nodi_artificiali, m, p=0.8)
print(f"Grafo finale: {G3.number_of_nodes()} nodi, {G3.number_of_edges()} archi.")

In [15]:
with open("graphs/grafo_artificiale_v2.json", "w") as f:
    json.dump(nx.node_link_data(G3), f, indent=4)

# <hr style="border: 1px solid white; margin-top: 10px;">

# ___Caricamento dei Grafi da JSON___
Invece di ricalcolare i grafi ogni volta, li carichiamo direttamente dai rispettivi file JSON.

In [ ]:
with open("data/grafo_diretto.json", "r") as f:
    data = json.load(f)

G = nx.node_link_graph(data)  
print(f"Grafo diretto: {G.number_of_nodes()} nodi, {G.number_of_edges()} archi.")

with open("graphs/grafo_artificiale.json", "r") as f:
    data = json.load(f)

G2 = nx.node_link_graph(data)  
print(f"Grafo artificiale: {G2.number_of_nodes()} nodi, {G2.number_of_edges()} archi.")

with open("graphs/grafo_artificiale_v2.json", "r") as f:
    data = json.load(f)

G3 = nx.node_link_graph(data)  
print(f"Grafo artificale v2: {G3.number_of_nodes()} nodi, {G3.number_of_edges()} archi.")

# <hr style="border: 1px solid white; margin-top: 10px;">

# ___Visualizzazione Grafi___

### ___Visualizzazione statica dei Grafi___
Ora visualizziamo i due grafi in maniera _statica_.

In [17]:
plt.figure(figsize=(15, 15), dpi=200)
pos = nx.spring_layout(G, iterations=300, k = 1.05)
nx.draw(G, pos, with_labels=True, font_size=5.5, node_size=30, node_color="red", edge_color="gray", width=0.1)
plt.savefig("grafici2d/grafico_diretto.png", dpi=200)
plt.close()

In [18]:
plt.figure(figsize=(15, 15), dpi=200)
pos = nx.spring_layout(G2, iterations=300, k = 2.5)
nx.draw(G2, pos, with_labels=True, font_size=5.5, node_size=30, node_color="red", edge_color="gray", width=0.1)
plt.savefig("grafici2d/grafico_artificiale.png", dpi=200)
plt.close()

In [19]:
# Calcola i gradi dei nodi del grafo G2
degree_sequence = [d for n, d in G2.degree()]
degree_counts = Counter(degree_sequence)

# Ordina i gradi e ottieni le frequenze
degrees, counts = zip(*sorted(degree_counts.items()))
counts = np.array(counts)

# Calcolo della frequenza cumulativa normalizzata
cumulative_counts = np.cumsum(counts[::-1])[::-1]  # Frequenza cumulativa
cumulative_probs = cumulative_counts / cumulative_counts[0]  # Normalizzazione

# Grafico log-log
plt.figure(figsize=(8, 6))
plt.loglog(degrees, cumulative_probs, marker="o", linestyle="none")
plt.title("Distribuzione cumulativa del grado (Preferential Attachment)")
plt.xlabel("Grado (d)")
plt.ylabel("Frequenza cumulativa normalizzata P(d)")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.savefig("grafici2d/power_law.png", dpi=200)
plt.close()

In [20]:
plt.figure(figsize=(15, 15), dpi=200)
pos = nx.spring_layout(G3, iterations=300, k = 2.5)
nx.draw(G3, pos, with_labels=True, font_size=5.5, node_size=30, node_color="red", edge_color="gray", width=0.1)
plt.savefig("grafici2d/grafico_artificiale_v2.png", dpi=200)
plt.close()

In [21]:
# Calcola i gradi dei nodi del grafo G3
degree_sequence = [d for n, d in G3.degree()]
degree_counts = Counter(degree_sequence)

# Ordina i gradi e ottieni le frequenze
degrees, counts = zip(*sorted(degree_counts.items()))
counts = np.array(counts)

# Calcolo della frequenza cumulativa normalizzata
cumulative_counts = np.cumsum(counts[::-1])[::-1]  # Frequenza cumulativa
cumulative_probs = cumulative_counts / cumulative_counts[0]  # Normalizzazione

# Grafico log-log
plt.figure(figsize=(8, 6))
plt.loglog(degrees, cumulative_probs, marker="o", linestyle="none")
plt.title("Distribuzione cumulativa del grado (Random Walk)")
plt.xlabel("Grado (d)")
plt.ylabel("Frequenza cumulativa normalizzata P(d)")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.savefig("grafici2d/power_law_v2.png", dpi=200)
plt.close()

### ___Visualizzazione interattiva dei Grafi___
I grafi sono visualizzati utilizzando la libreria _pyvis_ e sono salvati in formato HTML.
<br> I file risultanti sono i seguenti:
<br> -> _grafo_diretto_interattivo.html_
<br> -> _grafo_artificiale_interattivo.html_

In [ ]:
visualizza_interattivo_con_pyvis(G, "html/grafo_diretto_interattivo.html")

In [ ]:
visualizza_interattivo_con_pyvis(G2, "html/grafo_artificiale_interattivo.html")

In [ ]:
visualizza_interattivo_con_pyvis(G3, "html/grafo_artificiale_v2_interattivo.html")

# <hr style="border: 1px solid white; margin-top: 10px;">

# ___Calcolo delle metriche___
# Scelta per la serializzazione delle metriche

Per ciascun grafo (diretto e artificiale), vengono calcolate esclusivamente le metriche pertinenti al loro contesto specifico:
- **Grafo diretto**: Vengono calcolate metriche quali PageRank, In-degree e Out-degree, che dipendono dalla direzione degli archi. Le metriche globali come il centro e il raggio sono omesse qualora non applicabili (ad esempio, nel caso in cui il grafo non sia fortemente connesso).
- **Grafo artificiale (indiretto)**: In questo caso, vengono calcolate tutte le metriche globali (centro, raggio, distanza media, ecc.), mentre metriche come PageRank o In/Out-degree, che non sono rilevanti per un grafo indiretto, vengono omesse.

Qualora una metrica non sia pertinente, essa viene indicata come **null** o “non calcolabile”.


### __GRAFO DIRETTO__

In [ ]:
if nx.is_strongly_connected(G):
    print("Il grafo delle dipendenze è fortemente connesso.")
else:
    print("Il grafo delle dipendenze non è fortemente connesso.")

In [ ]:
if nx.is_connected(G3):
    print("Il grafo delle dipendenze è fortemente connesso.")
else:
    print("Il grafo delle dipendenze non è fortemente connesso.")

In [ ]:
componenti = list(nx.strongly_connected_components(G))
print(f"Numero di componenti fortemente connesse: {len(componenti)}")
print(f"Numero di nodi: {G.number_of_nodes()}")

_misure globali_

In [28]:
distanze = []
for nodo in G.nodes:
    lunghezze = nx.single_source_shortest_path_length(G, nodo)
    distanze.extend(lunghezze.values())

distanza_media = sum(distanze) / len(distanze)
distanza_massima = max(distanze)
clustering_medio = nx.average_clustering(G)
transitivita = nx.transitivity(G)

_misure di centralità_

In [29]:
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
degree_centrality = nx.degree_centrality(G)
in_degree_centrality = nx.in_degree_centrality(G)
out_degree_centrality = nx.out_degree_centrality(G)
page_rank = nx.pagerank(G)

centrality_measures = {}
for nodo in G.nodes:
    centrality_measures[nodo] = {
        "betweenness_centrality": betweenness_centrality.get(nodo, 0),
        "closeness_centrality": closeness_centrality.get(nodo, 0),
        "degree_centrality": degree_centrality.get(nodo, 0),
        "in_degree_centrality": in_degree_centrality.get(nodo, 0),
        "out_degree_centrality": out_degree_centrality.get(nodo, 0),
        "page_rank": page_rank.get(nodo, 0)
    }

In [ ]:
risultati = {
    "global_metrics": {
        "graph_center": ["non calcolabile"],
        "radius": "non calcolabile",
        "average_distance": distanza_media,
        "max_distance": distanza_massima,
        "clustering_coefficient": clustering_medio,
        "transitivity": transitivita,
    },
    "centrality_measures": centrality_measures,
    "small_worldness": "non calcolabile"
}

with open("misure/misure_grafo_diretto.json", "w") as f:
    json.dump(risultati, f, indent=4)

print("Analisi completa salvata in misure_grafo_diretto.json")

---

### ___GRAFO ARTIFICIALE___

_misure globali_

In [31]:
centro = nx.center(G2)
raggio = nx.radius(G2)
distanza_media = nx.average_shortest_path_length(G2)
distanza_massima = nx.diameter(G2)
clustering_medio = nx.average_clustering(G2)
transitivita = nx.transitivity(G2)

_misure di centralità_

In [32]:
betweenness_centrality = nx.betweenness_centrality(G2)
closeness_centrality = nx.closeness_centrality(G2)
degree_centrality = nx.degree_centrality(G2)

centrality_measures = {}
for nodo in G2.nodes:
    centrality_measures[nodo] = {
        "betweenness_centrality": betweenness_centrality.get(nodo, 0),
        "closeness_centrality": closeness_centrality.get(nodo, 0),
        "degree_centrality": degree_centrality.get(nodo, 0),
        "in_degree_centrality": degree_centrality.get(nodo, 0),
        "out_degree_centrality": degree_centrality.get(nodo, 0),
        "page_rank": "non calcolabile"
    }

In [ ]:
risultati = {
    "global_metrics": {
        "graph_center": centro,
        "radius": raggio,
        "average_distance": distanza_media,
        "max_distance": distanza_massima,
        "clustering_coefficient": clustering_medio,
        "transitivity": transitivita,
    },
    "centrality_measures": centrality_measures,
    "small_worldness": calcola_small_worldness(G2)
}

with open("misure/misure_grafo_artificiale.json", "w") as f:
    json.dump(risultati, f, indent=4)

print("Analisi completa salvata in misure_grafo_artificiale.json")

---

### ___GRAFO ARTIFICIALE RANDOM WALK___

_misure globali_

In [34]:
centro = nx.center(G3)
raggio = nx.radius(G3)
distanza_media = nx.average_shortest_path_length(G3)
distanza_massima = nx.diameter(G3)
clustering_medio = nx.average_clustering(G3)
transitivita = nx.transitivity(G3)

_misure di centralità_

In [35]:
betweenness_centrality = nx.betweenness_centrality(G3)
closeness_centrality = nx.closeness_centrality(G3)
degree_centrality = nx.degree_centrality(G3)

centrality_measures = {}
for nodo in G3.nodes:
    centrality_measures[nodo] = {
        "betweenness_centrality": betweenness_centrality.get(nodo, 0),
        "closeness_centrality": closeness_centrality.get(nodo, 0),
        "degree_centrality": degree_centrality.get(nodo, 0),
        "in_degree_centrality": degree_centrality.get(nodo, 0),
        "out_degree_centrality": degree_centrality.get(nodo, 0),
        "page_rank": "non calcolabile"
    }

In [ ]:
risultati = {
    "global_metrics": {
        "graph_center": centro,
        "radius": raggio,
        "average_distance": distanza_media,
        "max_distance": distanza_massima,
        "clustering_coefficient": clustering_medio,
        "transitivity": transitivita,
    },
    "centrality_measures": centrality_measures,
    "small_worldness": calcola_small_worldness(G3)
}

with open("misure/misure_grafo_artificiale_v2.json", "w") as f:
    json.dump(risultati, f, indent=4)

print("Analisi completa salvata in misure_grafo_artificiale_v2.json")